# SQL на python с нуля


<p style="align: center;"><img align=center src="https://www.freecodecamp.org/news/content/images/2020/08/Untitled-design-1-.png"  width=400></p>


In [ ]:
# Подробный разбор приведен тут https://habr.com/ru/post/321510/

In [3]:
import sqlite3 # Импортируем библиотеку, соответствующую типу нашей базы данных 

С базой можно работать и во всяких редакторах. Советую вот эту
# https://sqlitestudio.pl/  


<p style="align: center;"><img align=center src="https://habrastorage.org/files/38a/503/ff2/38a503ff27b846e4aac2411fb0fdf614.png"  width=400></p>


In [ ]:
Скачаем себе готовую базу

https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite

# Главный шаблон

In [91]:
# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

# ТУТ БУДЕТ НАШ КОД РАБОТЫ С БАЗОЙ ДАННЫХ
# КОД ДАЛЬНЕЙШИХ ПРИМЕРОВ ВСТАВЛЯТЬ В ЭТО МЕСТО

# Не забываем закрыть соединение с базой данных
#conn.close()

 но что если закрыть/не закрыть?

# Чтение из базы

In [97]:
# Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
cursor.execute("SELECT Name FROM Artist LIMIT 3")

 

# Получаем результат сделанного запроса
results = cursor.fetchall()
results2 =  cursor.fetchall()

print(results)   # [('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',), ('Aaron Goldberg',)]
print(results2)  # []


# Не забываем закрыть соединение с базой данных
#conn.close()

[('AC/DC',), ('Accept',), ('Aerosmith',)]
[]


# Слишком просто? Ну, так и должно быть!

<p style="align: center;"><img align=center src="https://lh3.googleusercontent.com/proxy/qlD_MrTxT8OWbvjyTCHq1I9N4a1hbxkX_zzVdufnm2R0iK9VFdMH6ZhDc4Qzzp4o2BF5G3-Q5CEvgNuetGamESF41nQ"  width=400></p>


Обратите внимание: После получения результата из курсора, второй раз без повторения самого запроса его получить нельзя — вернется пустой результат!

# Запись в базу

In [107]:
# Делаем INSERT запрос к базе данных, используя обычный SQL-синтаксис
#cursor.execute("insert into Artist values (Null, 'A Aagrh!') ")
cursor.execute("INSERT INTO Artist VALUES (Null, 'Los Angeles')")

 
conn.commit() # Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию


# Проверяем результат
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
results = cursor.fetchall()
print(results)  # [('A Aagrh!',), ('A Cor Do Som',), ('Aaron Copland & London Symphony Orchestra',)]

[('A Aagrh!',), ('A Aagrh!',), ('A Aagrh!',)]


Примечание: Если к базе установлено несколько соединений и одно из них осуществляет модификацю базы, то база SQLite залочивается до завершения (метод соединения .commit()) или отмены (метод соединения .rollback()) транзакции.

# Разбиваем запрос на несколько строк в тройных кавычках

Длинные запросы можно разбивать на несколько строк в произвольном порядке,

если они заключены в тройные кавычки — одинарные ('''…''') или двойные ("""...""")

In [108]:
sql = """
  SELECT name
  FROM Artist
  ORDER BY Name LIMIT 3
"""


cursor.execute(sql)

# Объединяем запросы к базе данных в один вызов метода


Метод курсора .execute() позволяет делать только один запрос за раз, при попытке сделать несколько через точку с запятой будет ошибка.

In [ ]:
#Правильно!
cursor.execute("""insert into Artist values (Null, 'A Aagrh!');""")
cursor.execute("""insert into Artist values (Null, 'A Aagrh-2!');""")

Для решения такой задачи можно либо несколько раз вызывать метод курсора .execute()

In [ ]:
#Неправильно!
cursor.execute("""
	insert into Artist values (Null, 'A Aagrh!');
	insert into Artist values (Null, 'A Aagrh-2!');
""")
# sqlite3.Warning: You can only execute one statement at a time.

Либо использовать метод курсора .executescript()

In [ ]:
cursor.executescript("""
 insert into Artist values (Null, 'A Aagrh!');
 insert into Artist values (Null, 'A Aagrh-2!');
""")

Данный метод также удобен, когда у нас запросы сохранены в отдельной переменной или даже в файле и нам его надо применить такой запрос к базе.

# Делаем множественную вставку строк проходя по коллекции с помощью метода курсора .executemany()

In [ ]:
# Обратите внимание, даже передавая одно значение - его нужно передавать кортежем!
# Именно по этому тут используется запятая в скобках!
new_artists = [
    ('A Aagrh!',),
    ('A Aagrh!-2',),
    ('A Aagrh!-3',),
]
cursor.executemany("insert into Artist values (Null, ?);", new_artists)

# Получаем результаты по одному, используя метод курсора .fetchone()

Он всегда возвращает кортеж или None. если запрос пустой.

In [ ]:
cursor.execute("SELECT Name FROM Artist ORDER BY Name LIMIT 3")
print(cursor.fetchone())    # ('A Cor Do Som',)
print(cursor.fetchone())    # ('Aaron Copland & London Symphony Orchestra',)
print(cursor.fetchone())    # ('Aaron Goldberg',)
print(cursor.fetchone())    # None

Важно! Стандартный курсор забирает все данные с сервера сразу, не зависимо от того, используем мы .fetchall() или .fetchone()

# Повышаем устойчивость кода
Для большей устойчивости программы (особенно при операциях записи) можно оборачивать инструкции обращения к БД в блоки «try-except-else» и использовать встроенный в sqlite3 «родной» объект ошибок, например, так:

In [ ]:
try:
    cursor.execute(sql_statement)
    result = cursor.fetchall()
except sqlite3.DatabaseError as err:       
    print("Error: ", err)
else:
    conn.commit()

# Пример использования в коде

In [9]:
# Импортируем библиотеку, соответствующую типу нашей базы данных 
import sqlite3

# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor() # Создаем курсор  


# Не забываем закрыть соединение с базой данных
#conn.close()

def create_table():
    cursor.execute("""CREATE TABLE tourists (u_id integer, m_text text, city text, date text) """)
    return 0

def get_Data(city):
    sql = "SELECT * FROM tourists WHERE city=%s" % city
    cursor.execute(sql)
    conn.commit()
    return cursor.fetchall()

def add_Data(id, text, city, date):
    cursor.execute("INSERT INTO tourists VALUES (?,?,?,?)", (id, text, city, date))
    conn.commit()
    
def Update_Data(group_id, i):                     # Добавляем балл
    req ="UPDATE tourists SET count=? WHERE group_id=?;"
    cursor.execute(req, (i, group_id))
    conn.commit()

# Эта ячейка не работает. Приведена просто в качестве демонстрации! 
# Эта ячейка не работает. Приведена просто в качестве демонстрации!
# Эта ячейка не работает. Приведена просто в качестве демонстрации!
# Эта ячейка не работает. Приведена просто в качестве демонстрации!


# А как же pandas? 
Простенькая альтернатива для простых проектов

In [109]:
import pandas as pd

# Создаем таблицу

In [110]:
DATA = pd.DataFrame(columns=['id','Имя', 'Текста'])
 
DATA.loc[len(DATA)] = [1, 'Александр', 'Какой-то ваш текст']
DATA.loc[len(DATA)] = [2, 'Дмитрий', 'Еще какой-то текст']
DATA.loc[len(DATA)] = [3, 'Андрей', 'И снова какой-то текст']

DATA

,id,Имя,Текста
0,1,Александр,Какой-то ваш текст
1,2,Дмитрий,Еще какой-то текст
2,3,Андрей,И снова какой-то текст


# Сохраняем таблицу

In [82]:
DATA.to_csv('DATA.csv')  # 0.68

# Снова открываем таблицу

In [111]:
Data_again = pd.read_csv('DATA.csv', index_col=0) # , index_col=0 !!!
Data_again

,id,Имя,Текста
0,1,Александр,Какой-то ваш текст
1,2,Дмитрий,Еще какой-то текст
2,3,Андрей,И снова какой-то текст


In [112]:
Data_again[Data_again.id==1]['Текста']

0    Какой-то ваш текст
Name: Текста, dtype: object

но все равно не SQL =(

# Теперь привяжем бд к боту!

In [124]:
conn = sqlite3.connect('data.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

In [126]:
def create_table():
    cursor.execute("""CREATE TABLE visitors_tg (tg_nik text, data text, sent_text text) """)
    return 0

create_table()

0

In [136]:
def insert_data(tg_nik, data, sent_text):
    conn = sqlite3.connect('data.sqlite')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO visitors_tg VALUES (?,?,?)", (tg_nik, data, sent_text))
    conn.commit()
    conn.close()

In [131]:
tg_nik, data, sent_text = 'some_nik', '230980234', 'some text'
insert_data(tg_nik, data, sent_text)

In [132]:
cursor.execute("SELECT * FROM visitors_tg ORDER BY tg_nik LIMIT 3")
results = cursor.fetchall()
print(results)  

[('some_nik', '230980234', 'some text')]


In [140]:
import random

import telebot
from telebot import types

Token = ' ваш токен бота'  # Ваш Токен из BotFather  У вас должен быть свой токен
bot = telebot.TeleBot(Token)


@bot.message_handler(commands=['start'])
def welcome(message):
    # keyboard
    
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("🎲 Рандомное число")
    item2 = types.KeyboardButton("😊 Как дела?")
    markup.add(item1, item2)
 
    bot.send_message(message.chat.id, "Добро пожаловать, {0.first_name}!\nЯ - <b>{1.first_name}</b>, бот созданный чтобы быть подопытным кроликом.".format(message.from_user, bot.get_me()),
        parse_mode='html', reply_markup=markup)


@bot.message_handler(content_types=['text'])
def lalala(message):
     
    name = message.from_user.username
    date = message.date
    text = message.text
    print(name, date, text)
    insert_data(name, date, text)
    
    if message.chat.type == 'private':
        if message.text == '🎲 Рандомное число':
            bot.send_message(message.chat.id, str(random.randint(0,100)))
        elif message.text == '😊 Как дела?':
 
            markup = types.InlineKeyboardMarkup(row_width=2)
            item1 = types.InlineKeyboardButton("Хорошо", callback_data='good')
            item2 = types.InlineKeyboardButton("Не очень", callback_data='bad')
 
            markup.add(item1, item2)
 
            bot.send_message(message.chat.id, 'Отлично, сам как?', reply_markup=markup)
        else:
            bot.send_message(message.chat.id, 'Я не знаю что ответить 😢')
            
            
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    try:
        if call.message:
            if call.data == 'good':
                bot.send_message(call.message.chat.id, 'Вот и отличненько 😊')
            elif call.data == 'bad':
                bot.send_message(call.message.chat.id, 'Бывает 😢')
 
            # remove inline buttons
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="😊 Как дела?",
                reply_markup=None)
 
            # show alert
            bot.answer_callback_query(callback_query_id=call.id, show_alert=False,
                text="ЭТО ТЕСТОВОЕ УВЕДОМЛЕНИЕ!!11")
 
    except Exception as e:
        print(repr(e))
 

# RUN, Вася, RUN!
bot.polling(none_stop=True)


# Время в Unix time формате

None 1607155208 None 1607155208 🎲 Рандомное число
😊 Как дела?
NoneNone 1607155211 😊 Как дела?
 1607155211 🎲 Рандомное число
None 1607155212 🎲 Рандомное число
None 1607155212 😊 Как дела?
None 1607155212 🎲 Рандомное число
ApiTelegramException('A request to the Telegram API was unsuccessful. Error code: 400 Description: Bad Request: message is not modified: specified new message content and reply markup are exactly the same as a current content and reply markup of the message')
None 1607155228 🎲 Рандомное число
None 1607155233 🎲 Рандомное число
None 1607155239 Приветик
agrlx 1607155240 Приветы
agrlx 1607155263 ывыв
WestEST77 1607155281 Привет!


In [142]:
cursor.execute("SELECT * FROM visitors_tg ORDER BY tg_nik")
results = cursor.fetchall()
print(results)  

[(None, '1607155161', '5'), (None, '1607155178', '🎲 Рандомное число'), (None, '1607155180', '😊 Как дела?'), (None, '1607155186', '🎲 Рандомное число'), (None, '1607155194', '🎲 Рандомное число'), (None, '1607155196', '😊 Как дела?'), (None, '1607155208', '😊 Как дела?'), (None, '1607155208', '🎲 Рандомное число'), (None, '1607155211', '😊 Как дела?'), (None, '1607155211', '🎲 Рандомное число'), (None, '1607155212', '🎲 Рандомное число'), (None, '1607155212', '😊 Как дела?'), (None, '1607155212', '🎲 Рандомное число'), (None, '1607155228', '🎲 Рандомное число'), (None, '1607155233', '🎲 Рандомное число'), (None, '1607155239', 'Приветик'), ('Aleron75infskin', '1607155038', '😊 Как дела?'), ('Aleron75infskin', '1607155152', '😊 Как дела?'), ('WestEST77', '1607155281', 'Привет!'), ('agrlx', '1607155240', 'Приветы'), ('agrlx', '1607155263', 'ывыв'), ('some_nik', '230980234', 'some text')]
